In [120]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
from pathlib import Path
import os.path
import time
from collections import OrderedDict
import csv
import shutil

In [121]:
def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d %H:%M:%S")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
def convert_date(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
def convert_to_datetime(struct_time):
    return datetime(*struct_time[:6])
def input_data(data_dir):
    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.*'):
        file_suffixes = filename.suffixes
        if file_suffixes[-1].lower() in ['.xlsx', '.csv']:
            export_time = os.path.getmtime(filename)
            export_time_datetime = convert_to_datetime(time.localtime(export_time))
            
            if file_suffixes[-1].lower() == '.xlsx':
                file_name = filename.stem
                dfs = pd.read_excel(filename, sheet_name=None, skiprows=0, na_values=None)
                for sheet_name, df in dfs.items():
                    df['sheet_name'] = file_name 
                    df['Export time'] = export_time_datetime
                    list_files.append(df)
                    
            elif file_suffixes[-1].lower() == '.csv':
                file_name = filename.stem
                df = pd.read_csv(filename, skiprows=0, na_values=None)
                df['sheet_name'] = file_name
                df['Export time'] = export_time_datetime
                list_files.append(df)
                
    df_list = pd.concat(list_files, axis=0, ignore_index=True)
    return df_list

In [122]:
# IEX_Input = input_data(r'C:\Users\huuchinh.nguyen\Concentrix Corporation\Thi Nhu Thao Phan - Expedia_VN_RTAs (shared drive)\EXPEDIA VN REPORT\Schedule\IEX Daily')
IEX_Input = input_data('IEX')


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [123]:
columns_to_remove = ["Unnamed: 0", "Unnamed: 8", "Unnamed: 9", "Unnamed: 5", "Unnamed: 11", "Unnamed: 12"]

IEX = IEX_Input[[col for col in IEX_Input.columns if col not in columns_to_remove]]
IEX = IEX.rename(columns={
    "Unnamed: 1": "Agent",
    "Unnamed: 2": "Date",
    "Unnamed: 3": "Start_Shift",
    "Unnamed: 4": "End_Shift",
    "Unnamed: 6": "Scheduled Activity",
    "Unnamed: 7": "Start_Action",
    "Unnamed: 10": "End_Action"
})

IEX['Generate Date'] = np.where(
    IEX['Agent'].str.contains("Generation Date: ", na=False),
    IEX['Agent'].str.extract(r'Generation Date: (.+)')[0],
    np.nan
)
IEX['Generate Date'] = IEX['Generate Date'].fillna(method='bfill')
IEX_edit = IEX.copy()
IEX_edit['Agent'] = IEX_edit['Agent'].fillna(method='ffill')
offTable_save = IEX_edit
IEX_edit = IEX_edit[(IEX_edit['Start_Shift'] != "Off") & (IEX_edit['Date'] != "Date")]
IEX_edit = IEX_edit[~(IEX_edit['Date'].isna() & IEX_edit['Scheduled Activity'].isna())]
IEX_edit

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20364\2893895032.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  IEX['Generate Date'] = IEX['Generate Date'].fillna(method='bfill')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20364\2893895032.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  IEX_edit['Agent'] = IEX_edit['Agent'].fillna(method='ffill')


,Agent,Date,Start_Shift,End_Shift,Scheduled Activity,Start_Action,End_Action,sheet_name,Export time,Generate Date
9,"Agent: 3005532 AMINE, AFRANDY",8/5/24,3:00 PM,12:00 AM,Open Time,3:00 PM,5:45 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
10,"Agent: 3005532 AMINE, AFRANDY",NaN,NaN,NaN,Break,5:45 PM,6:00 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
11,"Agent: 3005532 AMINE, AFRANDY",NaN,NaN,NaN,Open Time,6:00 PM,8:15 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
12,"Agent: 3005532 AMINE, AFRANDY",NaN,NaN,NaN,Lunch,8:15 PM,9:15 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
13,"Agent: 3005532 AMINE, AFRANDY",NaN,NaN,NaN,Open Time,9:15 PM,10:45 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
...,...,...,...,...,...,...,...,...,...,...
6613,"Agent: 3001034 tran, thihaiyen",NaN,NaN,NaN,Open Time,4:45 PM,6:00 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
6614,"Agent: 3001034 tran, thihaiyen",NaN,NaN,NaN,Lunch,6:00 PM,7:00 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
6615,"Agent: 3001034 tran, thihaiyen",NaN,NaN,NaN,Open Time,7:00 PM,10:00 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM
6616,"Agent: 3001034 tran, thihaiyen",NaN,NaN,NaN,Break,10:00 PM,10:15 PM,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,8/11/24 8:57 PM


In [124]:
offTable = offTable_save[offTable_save['Start_Shift'] == "Off"].reset_index(drop=True)

condition = (IEX_edit['Date'] != "Date") & (IEX_edit['Start_Shift'] != "Off")
IEX_edit = IEX_edit[condition].reset_index(drop=True)
IEX_edit[['Date', 'Start_Shift', 'End_Shift']] = IEX_edit[['Date', 'Start_Shift', 'End_Shift']].fillna(method='ffill')

condition2 = (IEX_edit['Agent'].str.contains("Agent: ", na=False)) | (IEX_edit['Agent'].isna())
IEX_edit = IEX_edit[condition2].reset_index(drop=True)

offTable['Scheduled Activity'] = offTable['Scheduled Activity'].fillna(offTable['Start_Shift'])
offTable['Start_Shift'] = offTable['Start_Shift'].replace('Off', np.nan)

IEX_edit = IEX_edit[IEX_edit['Scheduled Activity'].notna()].reset_index(drop=True)
combined_df = pd.concat([offTable, IEX_edit], axis=0, ignore_index=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20364\1879448329.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  IEX_edit[['Date', 'Start_Shift', 'End_Shift']] = IEX_edit[['Date', 'Start_Shift', 'End_Shift']].fillna(method='ffill')


In [125]:
combined_df['Export time'] = pd.to_datetime(combined_df['Export time'], errors='coerce')
combined_df['Generate Date'] = pd.to_datetime(combined_df['Generate Date'], errors='coerce')
combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
combined_df['Year'] = combined_df['Date'].dt.year
combined_df['Month'] = combined_df['Date'].dt.strftime('%b-%y')
combined_df['IEX_ID'] = combined_df['Agent'].str.extract(r'(\d+)', expand=False).astype(int)

max_generate_date_df = combined_df.groupby(['IEX_ID','Date'])['Generate Date'].max().reset_index()
max_generate_date_df = max_generate_date_df.rename(columns={'Generate Date': 'Max Generate Date'})

max_generate_date_with_details = pd.merge(combined_df, max_generate_date_df, on=['Date','IEX_ID'], how='left')
filtered_max_generate_date = max_generate_date_with_details[max_generate_date_with_details['Generate Date'] == max_generate_date_with_details['Max Generate Date']]

filtered_max_generate_date['Scheduled'] = filtered_max_generate_date.groupby(['Date', 'IEX_ID'])['Scheduled Activity'].transform(lambda x: 1 if any(x.isin(['Open Time', 'Extra Hours', 'No Call/No Show', 'PTO'])) else 0)
filtered_max_generate_date.to_excel('filtered_max_generate_date.xlsx')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20364\1031141896.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')


In [126]:
filtered_max_generate_date['Start_Shift'] = pd.to_datetime(filtered_max_generate_date['Start_Shift'], format='%I:%M %p', errors='coerce').dt.time
filtered_max_generate_date['End_Shift'] = pd.to_datetime(filtered_max_generate_date['End_Shift'], format='%I:%M %p', errors='coerce').dt.time
filtered_max_generate_date['Start_Action'] = pd.to_datetime(filtered_max_generate_date['Start_Action'], format='%I:%M %p', errors='coerce').dt.time
filtered_max_generate_date['End_Action'] = pd.to_datetime(filtered_max_generate_date['End_Action'], format='%I:%M %p', errors='coerce').dt.time

def create_datetime(row, time_column, shift_column=None):
    if row['Scheduled'] == 0:
        return pd.NaT
    if pd.isnull(row['Date']) or pd.isnull(row[time_column]):
        return pd.NaT
    time_value = row[time_column]
    if shift_column and pd.isnull(row[shift_column]):
        return pd.NaT
    if shift_column and time_value < row[shift_column]:
        return pd.Timestamp.combine(row['Date'], time_value) + pd.DateOffset(days=1)
    return pd.Timestamp.combine(row['Date'], time_value)

filtered_max_generate_date = filtered_max_generate_date.drop(filtered_max_generate_date[(filtered_max_generate_date['Scheduled Activity'].isin(['Lunch', 'Break','Training'])) & (filtered_max_generate_date['Scheduled'] == 0)].index)

filtered_max_generate_date['Datetime_Start_Action'] = filtered_max_generate_date.apply(lambda row: create_datetime(row, 'Start_Action', 'Start_Shift'), axis=1)
filtered_max_generate_date['Datetime_End_Action'] = filtered_max_generate_date.apply(lambda row: create_datetime(row, 'End_Action', 'Start_Shift'), axis=1)
filtered_max_generate_date.to_excel('Original Table.xlsx', index= True)

filtered_fluctuate_shifts = filtered_max_generate_date[
    filtered_max_generate_date['Scheduled Activity'].isin(["Open Time", "Extra Hours"]) | 
    filtered_max_generate_date['Scheduled Activity'].str.contains("Training", na=False)
]
min_max_fluctuate_shifts = filtered_fluctuate_shifts.groupby(['Date', 'IEX_ID']).agg(
    Datetime_Fluctuate_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_Fluctuate_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_fluctuate_shifts['Fluctuate Shift'] = (min_max_fluctuate_shifts['Datetime_Fluctuate_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_fluctuate_shifts['Datetime_Fluctuate_End_Shift'].dt.strftime('%H%M'))
new_table_with_fluctuate_shifts_col = pd.merge(filtered_max_generate_date, min_max_fluctuate_shifts, on=['Date', 'IEX_ID'], how='left')


min_max_first_shifts = filtered_max_generate_date.groupby(['Date', 'IEX_ID']).agg(
    Datetime_First_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_First_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_first_shifts['First Shift'] = (min_max_first_shifts['Datetime_First_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_first_shifts['Datetime_First_End_Shift'].dt.strftime('%H%M'))
new_table_with_first_shifts_col = pd.merge(new_table_with_fluctuate_shifts_col, min_max_first_shifts, on=['Date', 'IEX_ID'], how='left')

filtered_original_shifts = filtered_max_generate_date[
    filtered_max_generate_date['Scheduled Activity'].isin(["Open Time"]) | 
    filtered_max_generate_date['Scheduled Activity'].str.contains("Training", na=False)
]
min_max_original_shifts = filtered_original_shifts.groupby(['Date', 'IEX_ID']).agg(
    Datetime_Original_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_Original_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_original_shifts['Original Shift'] = (min_max_original_shifts['Datetime_Original_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_original_shifts['Datetime_Original_End_Shift'].dt.strftime('%H%M'))
new_table_with_original_shifts_col = pd.merge(new_table_with_first_shifts_col, min_max_original_shifts, on=['Date', 'IEX_ID'], how='left')

new_table_with_original_shifts_col = new_table_with_original_shifts_col[
    ~(
        (new_table_with_original_shifts_col['Scheduled Activity'].isin(["Lunch", "Break"])) & 
        new_table_with_original_shifts_col['Fluctuate Shift'].isna()
    )
]

new_table_with_original_shifts_col

,Agent,Date,Start_Shift,End_Shift,Scheduled Activity,Start_Action,End_Action,sheet_name,Export time,Generate Date,...,Datetime_End_Action,Datetime_Fluctuate_Start_Shift,Datetime_Fluctuate_End_Shift,Fluctuate Shift,Datetime_First_Start_Shift,Datetime_First_End_Shift,First Shift,Datetime_Original_Start_Shift,Datetime_Original_End_Shift,Original Shift
0,"Agent: 3005532 AMINE, AFRANDY",2024-08-08,NaT,NaT,Off,NaT,NaT,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
1,"Agent: 3005532 AMINE, AFRANDY",2024-08-11,NaT,NaT,Off,NaT,NaT,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
2,"Agent: 3001610 BUI, LYNHATDUY",2024-08-08,NaT,NaT,Off,NaT,NaT,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
3,"Agent: 3001610 BUI, LYNHATDUY",2024-08-09,NaT,NaT,Off,NaT,NaT,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
4,"Agent: 3003466 BUI, MINHTRIET",2024-08-10,NaT,NaT,Off,NaT,NaT,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5528,"Agent: 3001034 tran, thihaiyen",2024-08-10,15:00:00,00:00:00,Open Time,16:45:00,18:00:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,2024-08-10 18:00:00,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000
5529,"Agent: 3001034 tran, thihaiyen",2024-08-10,15:00:00,00:00:00,Lunch,18:00:00,19:00:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,2024-08-10 19:00:00,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000
5530,"Agent: 3001034 tran, thihaiyen",2024-08-10,15:00:00,00:00:00,Open Time,19:00:00,22:00:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,2024-08-10 22:00:00,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000
5531,"Agent: 3001034 tran, thihaiyen",2024-08-10,15:00:00,00:00:00,Break,22:00:00,22:15:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,2024-08-10 22:15:00,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000,2024-08-10 15:00:00,2024-08-11,1500-0000


In [127]:
sorted_generate_date = new_table_with_original_shifts_col.sort_values(by='Export time', ascending=True)

sorted_generate_date['Night_Shift'] = sorted_generate_date['Datetime_First_Start_Shift'].apply(lambda x: 1 if pd.to_datetime(x).hour >= 18 else 0)
sorted_generate_date['Week_Monday'] = sorted_generate_date['Date'] - pd.to_timedelta(sorted_generate_date['Date'].dt.weekday, unit='d')
sorted_generate_date['Week Begin'] = sorted_generate_date['Week_Monday'].dt.strftime('WB%d%m')
sorted_generate_date['Agent Name'] = sorted_generate_date['Agent'].str.extract(r'\d+ (.+)', expand=False).str.upper()
sorted_generate_date['Duration'] = (sorted_generate_date['Datetime_End_Action'] - sorted_generate_date['Datetime_Start_Action']).dt.total_seconds()

def calculate_total_time(df, activity_list, time_column_name):
    time_totals = df[df['Scheduled Activity'].isin(activity_list)]
    time_totals = time_totals.groupby(['Date', 'IEX_ID'])['Duration'].sum().reset_index()
    time_totals.rename(columns={'Duration': time_column_name}, inplace=True)
    time_totals[time_column_name] = time_totals[time_column_name] / 3600
    return time_totals

open_time_totals = calculate_total_time(sorted_generate_date, ['Open Time'], 'Open Time')
sorted_generate_date = sorted_generate_date.merge(open_time_totals, on=['Date', 'IEX_ID'], how='left')

rest_time_totals = calculate_total_time(sorted_generate_date, ['Break', 'Lunch'], 'Rest Time')
sorted_generate_date = sorted_generate_date.merge(rest_time_totals, on=['Date', 'IEX_ID'], how='left')

extra_time_totals = calculate_total_time(sorted_generate_date, ['Extra Hours'], 'Extra Time')
sorted_generate_date = sorted_generate_date.merge(extra_time_totals, on=['Date', 'IEX_ID'], how='left')

ncns_time_totals = calculate_total_time(sorted_generate_date, ['No Call/No Show'], 'NCNS')
sorted_generate_date = sorted_generate_date.merge(ncns_time_totals, on=['Date', 'IEX_ID'], how='left')

training_time_totals = calculate_total_time(sorted_generate_date, ['Training Offline','Training','Training Nesting'], 'Training')
sorted_generate_date = sorted_generate_date.merge(training_time_totals, on=['Date', 'IEX_ID'], how='left')

sorted_generate_date.to_excel('sorted_export_time.xlsx', index= True)

sorted_generate_date = sorted_generate_date.sort_values(by=['Date', 'IEX_ID', 'Datetime_Start_Action'])
sorted_generate_date['First_Scheduled_Activity'] = sorted_generate_date.groupby(['Date', 'IEX_ID'])['Scheduled Activity'].transform('first')

sorted_generate_date['Shift Tracking'] = np.where(
    (sorted_generate_date['Open Time'] > 0) & (sorted_generate_date['NCNS'] > 3),
    "HDL",
    np.where(
        ((sorted_generate_date['Open Time'] > 0) | (sorted_generate_date['Open Time'].isna())) & (sorted_generate_date['NCNS'] > 3),
        "UPL",
        np.where(
            (sorted_generate_date['Open Time'] > 0) & (sorted_generate_date['Extra Time'] > 0),
            "PR - OT",
            np.where(
                (sorted_generate_date['Open Time'] == 0) & (sorted_generate_date['Extra Time'] > 0),
                "PO",
                np.where(
                    (sorted_generate_date['Open Time'] > 0),
                    "PR",
                    np.where(
                        sorted_generate_date['First_Scheduled_Activity'] == "Extra Hours",
                        np.where(
                            sorted_generate_date['Open Time'].isna(),
                            "PO",
                            np.nan
                        ),
                        np.where(
                            sorted_generate_date['First_Scheduled_Activity'].isin(["Holiday", "Bereavement", "Off", "Off Phone Misc", "Termination", "Unscheduled"]),
                            sorted_generate_date['First_Scheduled_Activity'],
                            np.where(
                                sorted_generate_date['First_Scheduled_Activity'] == "PTO",
                                "AL",
                                np.where(
                                    sorted_generate_date['First_Scheduled_Activity'] == "Sickness",
                                    "SL",
                                    np.nan
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)

In [128]:
sorted_generate_date

,Agent,Date,Start_Shift,End_Shift,Scheduled Activity,Start_Action,End_Action,sheet_name,Export time,Generate Date,...,Week Begin,Agent Name,Duration,Open Time,Rest Time,Extra Time,NCNS,Training,First_Scheduled_Activity,Shift Tracking
1168,"Agent: 3000010 NGUYEN, THITHUYDUNG",2024-08-05,21:00:00,06:00:00,PTO,21:00:00,06:00:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"NGUYEN, THITHUYDUNG",32400.0,NaN,NaN,NaN,NaN,NaN,PTO,AL
4894,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Open Time,06:00:00,07:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"LE, MINHTRI",5400.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR
4893,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Break,07:30:00,07:45:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"LE, MINHTRI",900.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR
4892,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Open Time,07:45:00,09:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"LE, MINHTRI",6300.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR
4891,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Lunch,09:30:00,10:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"LE, MINHTRI",3600.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Open Time,09:15:00,11:15:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"TA, KHANHHOA",7200.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR
469,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Lunch,11:15:00,12:15:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"TA, KHANHHOA",3600.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR
468,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Open Time,12:15:00,14:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"TA, KHANHHOA",8100.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR
467,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Break,14:30:00,14:45:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,WB0508,"TA, KHANHHOA",900.0,7.5,1.5,NaN,NaN,NaN,Open Time,PR


In [129]:
def calculate_ot_pre_post_shift(df):
    extra_hours_df = df[df['Scheduled Activity'] == 'Extra Hours']
    
    ot_df = df[['Date', 'IEX_ID']].drop_duplicates()
    
    def calculate_duration_for_group(group_df, date, iex_id, shift_start, shift_end):
        pre_shift_duration = extra_hours_df[
            (extra_hours_df['Date'] == date) & 
            (extra_hours_df['IEX_ID'] == iex_id) & 
            (extra_hours_df['Datetime_Start_Action'] < shift_start)
        ]['Duration'].sum()
        
        post_shift_duration = extra_hours_df[
            (extra_hours_df['Date'] == date) & 
            (extra_hours_df['IEX_ID'] == iex_id) & 
            (extra_hours_df['Datetime_End_Action'] > shift_end)
        ]['Duration'].sum()
        
        return pre_shift_duration, post_shift_duration
    
    ot_df[['OT PreShift Seconds', 'OT PostShift Seconds']] = ot_df.apply(
        lambda row: pd.Series(
            calculate_duration_for_group(
                extra_hours_df, 
                row['Date'], 
                row['IEX_ID'], 
                df[(df['Date'] == row['Date']) & (df['IEX_ID'] == row['IEX_ID'])]['Datetime_Original_Start_Shift'].min(),
                df[(df['Date'] == row['Date']) & (df['IEX_ID'] == row['IEX_ID'])]['Datetime_Original_End_Shift'].max()
            )
        ), 
        axis=1
    )
    ot_df['OT PreShift'] = ot_df['OT PreShift Seconds'] / 3600
    ot_df['OT PostShift'] = ot_df['OT PostShift Seconds'] / 3600
    
    ot_df = ot_df.drop(columns=['OT PreShift Seconds', 'OT PostShift Seconds'])
    df = df.merge(ot_df, on=['Date', 'IEX_ID'], how='left')
    
    return df

OT_Pre_Post_Table = calculate_ot_pre_post_shift(sorted_generate_date)
OT_Pre_Post_Table = OT_Pre_Post_Table.groupby(['Date', 'IEX_ID'])[['OT PreShift', 'OT PostShift']].max().reset_index()

combine_OT_Pre_Post_Table = pd.merge(sorted_generate_date, OT_Pre_Post_Table, on=['Date','IEX_ID'], how='left')
combine_OT_Pre_Post_Table['Start Date'] = pd.to_datetime(combine_OT_Pre_Post_Table['Datetime_Start_Action']).dt.normalize()
combine_OT_Pre_Post_Table['End Date'] = pd.to_datetime(combine_OT_Pre_Post_Table['Datetime_End_Action']).dt.normalize()

combine_OT_Pre_Post_Table

,Agent,Date,Start_Shift,End_Shift,Scheduled Activity,Start_Action,End_Action,sheet_name,Export time,Generate Date,...,Rest Time,Extra Time,NCNS,Training,First_Scheduled_Activity,Shift Tracking,OT PreShift,OT PostShift,Start Date,End Date
0,"Agent: 3000010 NGUYEN, THITHUYDUNG",2024-08-05,21:00:00,06:00:00,PTO,21:00:00,06:00:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,NaN,NaN,NaN,NaN,PTO,AL,0.0,0.0,2024-08-05,2024-08-06
1,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Open Time,06:00:00,07:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-05,2024-08-05
2,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Break,07:30:00,07:45:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-05,2024-08-05
3,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Open Time,07:45:00,09:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-05,2024-08-05
4,"Agent: 3000124 LE, MINHTRI",2024-08-05,06:00:00,15:00:00,Lunch,09:30:00,10:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-05,2024-08-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Open Time,09:15:00,11:15:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-11,2024-08-11
5527,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Lunch,11:15:00,12:15:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-11,2024-08-11
5528,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Open Time,12:15:00,14:30:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-11,2024-08-11
5529,"Agent: 3085158 TA, KHANHHOA",2024-08-11,07:00:00,16:00:00,Break,14:30:00,14:45:00,IEX_05_Aug_11_Aug_2024,2024-08-12 15:58:06,2024-08-11 20:57:00,...,1.5,NaN,NaN,NaN,Open Time,PR,0.0,0.0,2024-08-11,2024-08-11


In [130]:
# Select the desired columns
iex_ids_with_max_export_time = combine_OT_Pre_Post_Table[['sheet_name', 'Generate Date','Year','Month','Week Begin','Date','Start Date','End Date','Agent Name', 'IEX_ID','Datetime_Fluctuate_Start_Shift',
       'Datetime_Fluctuate_End_Shift', 'Fluctuate Shift',
       'Datetime_First_Start_Shift', 'Datetime_First_End_Shift',
       'First Shift','Scheduled Activity','Datetime_Start_Action','Datetime_End_Action','Duration','Open Time','Extra Time','Rest Time','Training','NCNS','Night_Shift', 'Scheduled','Shift Tracking','OT PreShift', 'OT PostShift']]

sorted_df = iex_ids_with_max_export_time.sort_values(
    by=[ 'Date','IEX_ID','Datetime_Start_Action'],
    ascending=[ True, True,True]
).reset_index(drop=True)

sorted_df = sorted_df.drop_duplicates()

def label_breaks(group):
    group.loc[group['Scheduled Activity'] == 'Break', 'Scheduled Activity'] = (
        group['Scheduled Activity'] + '_' + (group['Scheduled Activity'].eq('Break').cumsum()).astype(str)
    )
    return group

sorted_df = sorted_df.groupby(['Date', 'IEX_ID']).apply(label_breaks)
sorted_df = sorted_df.reset_index(drop=True)

sorted_df

,sheet_name,Generate Date,Year,Month,Week Begin,Date,Start Date,End Date,Agent Name,IEX_ID,...,Open Time,Extra Time,Rest Time,Training,NCNS,Night_Shift,Scheduled,Shift Tracking,OT PreShift,OT PostShift
0,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-05,2024-08-05,2024-08-06,"NGUYEN, THITHUYDUNG",3000010,...,NaN,NaN,NaN,NaN,NaN,1,1,AL,0.0,0.0
1,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-05,2024-08-05,2024-08-05,"LE, MINHTRI",3000124,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0
2,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-05,2024-08-05,2024-08-05,"LE, MINHTRI",3000124,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0
3,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-05,2024-08-05,2024-08-05,"LE, MINHTRI",3000124,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0
4,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-05,2024-08-05,2024-08-05,"LE, MINHTRI",3000124,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5525,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-11,2024-08-11,2024-08-11,"TA, KHANHHOA",3085158,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0
5526,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-11,2024-08-11,2024-08-11,"TA, KHANHHOA",3085158,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0
5527,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-11,2024-08-11,2024-08-11,"TA, KHANHHOA",3085158,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0
5528,IEX_05_Aug_11_Aug_2024,2024-08-11 20:57:00,2024,Aug-24,WB0508,2024-08-11,2024-08-11,2024-08-11,"TA, KHANHHOA",3085158,...,7.5,NaN,1.5,NaN,NaN,0,1,PR,0.0,0.0


In [131]:
# Select and process relevant columns
split_table = sorted_df[['Date', 'Start Date', 'End Date', 'IEX_ID', 'Agent Name', 'Scheduled Activity', 'Datetime_Start_Action', 'Datetime_End_Action']]

# Calculate start and end time in hours
for col in ['Start', 'End']:
    split_table[f'{col}_Time_Hours'] = split_table[f'Datetime_{col}_Action'].dt.hour + \
                                       split_table[f'Datetime_{col}_Action'].dt.minute / 60 + \
                                       split_table[f'Datetime_{col}_Action'].dt.second / 3600
    split_table[f'{col}_Time_Hours'] = split_table[f'{col}_Time_Hours'].round(2)

# Adjust end time if it is less than start time
split_table['End_Time_Hours'] = split_table.apply(
    lambda row: row['End_Time_Hours'] + 24 if row['End_Time_Hours'] < row['Start_Time_Hours'] else row['End_Time_Hours'],
    axis=1
)

# Round times to the nearest half hour
for col in ['Start', 'End']:
    split_table[f'{col}_Time_Half_Rounding'] = split_table[f'{col}_Time_Hours'].apply(
        lambda x: np.round(x * 2) / 2 if np.round(x * 2) / 2 == x else np.floor(x * 2) / 2
    )

# Calculate number of splits
split_table['Number_Split'] = split_table.apply(
    lambda row: (row['End_Time_Half_Rounding'] - row['Start_Time_Half_Rounding']) * 2 
    if pd.notna(row['End_Time_Half_Rounding']) and pd.notna(row['Start_Time_Half_Rounding']) 
    else np.nan,
    axis=1
)

# Prepare for explosion
split_table = split_table.rename(columns={'Start_Time_Half_Rounding': 'Time_1'})
split_table['Time_2'] = split_table['Time_1'] + 0.5
split_table['Custom'] = split_table['Number_Split'].apply(lambda n: list(range(int(n) + 1)) if pd.notna(n) else [])
split_table = split_table.explode('Custom')

# Calculate new times
split_table['Time_1_New'] = split_table['Time_1'] + (split_table['Custom'] * 0.5)
split_table['Time_2_New'] = split_table['Time_2'] + (split_table['Custom'] * 0.5)
split_table['Rounded_End_Time_Hours'] = np.ceil(split_table['End_Time_Hours'] * 2) / 2

# Filter based on rounded end time
filtered_df = split_table[split_table['Rounded_End_Time_Hours'] > split_table['Time_1_New']]

# Calculate start and end time numbers
split_table['Start_Time_Number'] = np.where(
    split_table['Time_1_New'] > split_table['Start_Time_Hours'],
    split_table['Time_1_New'],
    split_table['Start_Time_Hours']
)

split_table['End_Time_Number'] = np.where(
    split_table['Time_2_New'] < split_table['End_Time_Hours'],
    split_table['Time_2_New'],
    split_table['End_Time_Hours']
)

# Calculate new day start and end times
def calculate_new_day_time(row, time_col, date_col):
    calculated_value = row[time_col]
    if row['Start Date'] == row['End Date']:
        return row[date_col]
    else:
        if time_col == 'Start_Time_Number' and calculated_value >= 24:
            return row['Start Date'] + pd.Timedelta(days=1)
        elif time_col == 'End_Time_Number' and calculated_value < 24:
            return row['End Date'] - pd.Timedelta(days=1)
        else:
            return row[date_col]

split_table['New_Day_Start_Time'] = split_table.apply(lambda row: calculate_new_day_time(row, 'Start_Time_Number', 'Start Date'), axis=1)
split_table['New_Day_End_Time'] = split_table.apply(lambda row: calculate_new_day_time(row, 'End_Time_Number', 'End Date'), axis=1)

# Adjust time numbers to fit within a 24-hour format
for col in ['Start', 'End']:
    split_table[f'{col}_Time_Number'] = (split_table[f'{col}_Time_Number'] - 24 * (split_table[f'{col}_Time_Number'] >= 24)) % 24

# Convert time numbers to actual time
def number_to_time(number):
    if pd.isna(number):
        return pd.NaT
    hours = int(np.floor(number))
    minutes = int(np.floor((number - hours) * 60))
    seconds = int(np.floor((number - hours - minutes / 60) * 3600))
    return pd.Timestamp(f"1970-01-01 {hours:02}:{minutes:02}:{seconds:02}")

for col in ['Start', 'End']:
    split_table[f'{col}_Time'] = split_table[f'{col}_Time_Number'].apply(number_to_time).dt.time

# Combine date and time into datetime
def combine_date_time(row, time_col, date_col):
    if pd.isna(row[time_col]):
        return pd.NaT
    return datetime.combine(row[date_col].date(), row[time_col])

split_table['Datetime_Start_Time'] = split_table.apply(lambda row: combine_date_time(row, 'Start_Time', 'New_Day_Start_Time'), axis=1)
split_table['Datetime_End_Time'] = split_table.apply(lambda row: combine_date_time(row, 'End_Time', 'New_Day_End_Time'), axis=1)

# Filter and save the final result
split_table = split_table[split_table['Datetime_Start_Time'] < split_table['Datetime_End_Time']]
split_table = split_table[['Date', 'Start Date', 'End Date', 'IEX_ID', 'Agent Name', 'Scheduled Activity', 'Datetime_Start_Action', 'Datetime_End_Action', 'Datetime_Start_Time', 'Datetime_End_Time']]
split_table.to_excel('Intervals_Extend.xlsx')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20364\1516825790.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_table[f'{col}_Time_Hours'] = split_table[f'Datetime_{col}_Action'].dt.hour + \
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20364\1516825790.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_table[f'{col}_Time_Hours'] = split_table[f'{col}_Time_Hours'].round(2)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20364\1516825790.py:6: SettingWithCopyWarning: 
A value is trying to be set on a

In [132]:
sorted_df['Date'] = sorted_df['Date'].dt.date
sorted_df.to_excel('test.xlsx', index=True)